In [ ]:
%pwd 

In [ ]:
# %cd /home/ubu-admin/Developer/tensorflow-wavelets
%cd /workspaces/OpenDVCW
import OpenDVCW
import numpy as np
import load
import tensorflow as tf
import matplotlib.pyplot as plt
import DataGen
import Callbacks
import datetime

In [ ]:
BATCH_SIZE = 1
EPOCHS = 700
STEPS_PER_EPOCH = 200
Height = 240
Width = 240
Channel = 3
lmbda = 2048
lr_init = 1e-4
early_stop = 15
I_QP=27

args = OpenDVCW.Arguments()
last = 4
checkponts_last_path = "checkpoints_wavelets_L_{}_{}_{}x{}/".format(lmbda, last, Width, Height)
checkponts_new_path = "checkpoints_wavelets_L_{}_{}_{}x{}/".format(lmbda, last+1, Width, Height)
save_name = "model_save_wavelets_L__{}_{}_{}x{}".format(lmbda, last+1, Width, Height)


In [ ]:
%load_ext tensorboard


In [ ]:
rm -rf ./logs/

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
model = OpenDVCW.OpenDVC(width=Width, height=Height, batch_size=BATCH_SIZE, num_filters=128, lmbda=lmbda)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_init),)
print("* [Model compiled]...")

print("* [Loading dataset]...")
data = DataGen.DataVimeo90kGenerator("folder_cloud_test.npy", 
                                    BATCH_SIZE,
                                    (Height,Width,Channel),
                                    Channel,
                                    True, 
                                    I_QP,
                                    True)

# print("Loading weights")
# model.load_weights(checkponts_last_path)


In [ ]:
# model.layers[0].trainable = False
# model.layers[1].trainable = False
# model.layers[2].trainable = True
# model.layers[3].trainable = True
# model.layers[4].trainable = False
# model.layers[5].trainable = True

In [ ]:
for layer in model.layers:
    print(layer.name, layer.trainable)

In [ ]:
# tf.config.run_functions_eagerly(True)
hist = model.fit(x=data, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS, verbose=1, batch_size=BATCH_SIZE,
                callbacks=[
                    Callbacks.MemoryCallback(),
                    Callbacks.LearningRateReducer(),
                    tf.keras.callbacks.ModelCheckpoint(filepath=checkponts_new_path, save_weights_only=True, save_freq='epoch', monitor="loss", mode='min',  save_best_only=True, verbose=1), 
                    tf.keras.callbacks.TerminateOnNaN(),
                    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=early_stop),
                    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, update_freq="epoch"),            
                    ],
				)

In [ ]:
path = load.load_random_path("folder_cloud_test.npy")
i=0
out_bin = "Test_com/test{}.bin".format(i)
out_decom = "Test_com/testdcom{}.png".format(i)
p_on_test = "Test_com/test_p_frame{}.png".format(i)
i_on_test = "Test_com/test_i_frame{}.png".format(i)

i_frame = path + 'im1' + '.png'
p_frame = path + 'im2' + '.png'
print(i_frame)

OpenDVCW.write_png(p_on_test, OpenDVCW.read_png_crop(p_frame, 240, 240))
OpenDVCW.write_png(i_on_test, OpenDVCW.read_png_crop(i_frame, 240, 240))

OpenDVCW.compress(model, i_frame, p_frame, out_bin, 240, 240)
OpenDVCW.decompress(model, i_frame, out_bin, out_decom, 240, 240)

In [ ]:
model.save(save_name, save_format="tf")